In [2]:
import requests, time, re, os, json
import pymysql, pyodbc
import sqlalchemy as sal
from sqlalchemy import create_engine
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from bs4 import BeautifulSoup as bs
from datetime import date

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#importing credentials
with open('../../../Notes-General/config.txt', 'r') as f:
    creds = f.read()
creds = json.loads(creds)
league = 'nfl'

# import credentials
dbUser = creds['mysqlSurface']['users'][1]
dbPw = creds['mysqlSurface']['creds']['jb']
dbHost = creds['mysqlSurface']['dbNFL']['host']
dbName = creds['mysqlSurface']['dbNFL']['database']
dbConnectionString = creds['pymysql'][league]

today = date.today()
season = 2024
week = int(input("What week is it >>>>>> ? "))
if week < 10:
    strWeek = "0" + str(week)
else:
    strWeek = week

What week is it >>>>>> ?  1


In [3]:
teammap = {'ATL': 'ATL',
 'BUF': 'BUF',
 'CHI': 'CHI',
 'CIN': 'CIN',
 'CLE': 'CLE',
 'DAL': 'DAL',
 'DEN': 'DEN',
 'DET': 'DET',
 'GB': 'GB',
 'TEN': 'TEN',
 'IND': 'IND',
 'KC': 'KC',
 'LV': 'LV',
 'LAR': 'LA',
 'MIA': 'MIA',
 'MIN': 'MIN',
 'NE': 'NE',
 'NO': 'NO',
 'NYG': 'NYG',
 'NYJ': 'NYJ',
 'PHI': 'PHI',
 'ARI': 'ARI',
 'PIT': 'PIT',
 'LAC': 'LAC',
 'SF': 'SF',
 'SEA': 'SEA',
 'TB': 'TB',
 'WSH': 'WAS',
 'CAR': 'CAR',
 'JAX': 'JAX',
 'BAL': 'BAL',
 'HOU': 'HOU',
 'FA': 'FA',
 'STL': 'SL',
 'SD': 'SD',
 'OAK': 'OAK'}
teammapid = {'ATL': '3800',
 'BUF': '0610',
 'CHI': '0810',
 'CIN': '0920',
 'CLE': '1050',
 'DAL': '1200',
 'DEN': '1400',
 'DET': '1540',
 'GB': '1800',
 'TEN': '2100',
 'IND': '2200',
 'KC': '2310',
 'LV': '2520',
 'LAR': '2510',
 'MIA': '2700',
 'MIN': '3000',
 'NE': '3200',
 'NO': '3300',
 'NYG': '3410',
 'NYJ': '3430',
 'PHI': '3700',
 'ARI': '3800',
 'PIT': '3900',
 'LAC': '4400',
 'SF': '4500',
 'SEA': '4600',
 'TB': '4900',
 'WSH': '5110',
 'CAR': '0750',
 'JAX': '2250',
 'BAL': '0325',
 'HOU': '2120',
 'FA': '0',
 'STL': '0',
 'SD': '0',
 'OAK': '0'}

In [ ]:
# creating lookup dictionaries that will be used across multiple database inserts
try:
    engine = create_engine(dbConnectionString)
    conn = engine.connect()
    
    # getting outlet db ids to convert the scraped names/ids
    outletLookup = pd.read_sql("SELECT outletId, outletName  FROM outlet;", conn)
    outletLookup = pd.Series(outletLookup.outletId.values, index=outletLookup.outletName).to_dict()
    
    # getting team db ids to convert datasource names to the ids
    #teamLookup = pd.read_sql("SELECT teamId, name  FROM team;", conn)
    teamLookup = pd.read_sql("SELECT teamId, nflfastrName  FROM team;", conn)
    teams = pd.read_sql("SELECT *  FROM team;", conn)
    teamLookup = pd.Series(teamLookup.teamId.values, index=teamLookup.nflfastrName).to_dict()
    
    
    # getting expert db ids to convert the scraped names/ids
    expertLookup = pd.read_sql("SELECT analystId, analystName  FROM analyst;", conn)
    expertLookup = pd.Series(expertLookup.analystId.values, index=expertLookup.analystName).to_dict()

    # getting pos db ids for espn
    posLookup = pd.read_sql("SELECT posId, pos  FROM pos;", conn)
    posLookup = pd.Series(posLookup.posId.values, index=posLookup.pos).to_dict()

    # create name table back up
    names = pd.read_sql(con=conn, sql="SELECT * FROM player")
    names.to_csv("../data/names_backup.csv", index=False)

    dbPlayers = pd.read_sql("SELECT joinName, team, position FROM players WHERE season = 2023;", conn)
    dbPlayers = pd.read_sql("SELECT * FROM players;", conn)

    conn.close()
    
    
except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)

# ADDING NEW PLAYERS

# LOAD RANKINGS

In [ ]:
try:
    #main df to hold all data
    df_load_rank = pd.DataFrame(columns = [
        'outlet', 'date', 'season', 'week', 'group', 'expert', 'rank', 'high', 'low','playerId'
    ])

    # df to hold players that are not in the database for a source yet
    df_missing_players_rank = pd.DataFrame(columns=['date', 'outlet', 'group', 'playerId', 'sourceId', 'name'])

    engine = create_engine(dbConnectionString)
    with engine.connect() as conn:

        # combining all outlets rankings to a single dataframe and converting names to the database Ids
        directory = r'../data/ranking/weekly'
        #looping through every rank file to aggregate into single df
        for filename in os.listdir(directory):
            f = os.path.join(directory,filename)
            # checking if it is a file
            if os.path.isfile(f):
                temp = pd.read_csv(f, parse_dates=['date'], infer_datetime_format=True,
                                   ###################################################################
                                   names=['outlet','date', 'season', 'week', 'group', 'expert', 'rank', 
                                          'name', 'sourceId', 'team', 'pos', 'high', 'low', 'playerId'],
                          skiprows=1)
                            ##########################################################################

                # updtaing outlet specific playerIds to database IDs
                if 'cbs_' in f:
                    lookup = pd.read_sql("SELECT playerId, cbsId FROM player WHERE cbsId IS NOT NULL;", conn)
                    lookup = pd.Series(lookup['playerId'].values, index=lookup.cbsId).to_dict()

                elif 'espn_' in f:
                    lookup = pd.read_sql("SELECT playerId, espnId FROM player WHERE espnId IS NOT NULL;", conn)
                    lookup = pd.Series(lookup.playerId.values, index=lookup.espnId).to_dict()

                elif ('fp_' in f) or ('fpEcr_' in f):
                    lookup = pd.read_sql("SELECT playerId, fpId FROM player WHERE fpId IS NOT NULL;", conn)
                    lookup = pd.Series(lookup.playerId.values, index=lookup.fpId).to_dict()

                elif 'nfl_' in f:
                    lookup = pd.read_sql("SELECT playerId, nflId FROM player WHERE nflId IS NOT NULL;", conn)
                    lookup['nflId'] = pd.to_numeric(lookup['nflId']) 
                    lookup = pd.Series(lookup.playerId.values, index=lookup.nflId).to_dict()

                # using the lookup to make the change from outletId to dbId
                temp['playerId'] = temp['sourceId'].map(lookup)

                ####################################
                # creating a df to hold that date for players who are not in the player table for the source
                if temp[pd.isnull(temp['playerId'])].shape[0] > 0:
                    df_missing_players_rank = pd.concat(
                                                    [df_missing_players_rank, 
                                                    temp.loc[
                                                             pd.isnull(temp['playerId']), 
                                                             ['date', 'outlet', 'group', 
                                                              'playerId', 'sourceId', 'name']
                                                        ]
                                                    ]
                                            )
                    df_missing_players_rank['joinName'] = df_missing_players_rank['name'].str.lower()
                    df_missing_players_rank['joinName'] = df_missing_players_rank['joinName'].str.replace(".", "")
                    df_missing_players_rank['joinName'] = df_missing_players_rank['joinName'].str.replace("''", "")
                    df_missing_players_rank['joinName'] = df_missing_players_rank['joinName'].str.replace("`", "")
                    df_missing_players_rank['joinName'] = df_missing_players_rank['joinName'].str.replace("", "")
                    
                    # grab the player data from nflfastR
                    dbPlayers = pd.read_sql("SELECT joinName, team, position FROM players ;", conn)
                    
                    # processing data
                    dbPlayers['position'] = dbPlayers['position'].replace(posLookup)
                    dbPlayers['team'] = dbPlayers['team'].replace(teamLookup)

                    
                    # adding position and team id to missing data
                    df_missing_players_rank = df_missing_players_rank.join(dbPlayers, on='joinName', how='left')
                    
                
                ####################################

                # updating outlet name to db outlet id 
                temp['outlet'] = temp['outlet'].replace(outletLookup)

                #updating expert name to db expert id
                temp['expert'] = temp['expert'].replace(expertLookup)

                temp = temp[['outlet', 'date', 'season', 'week', 'group', 'expert', 'rank', 'high', 'low', 'playerId']]
                # adding outlet dataframe to the upload dataframe
                df_load_rank = pd.concat([df_load_rank, temp])

                
    df_load_rank = df_load_rank.replace(np.nan, None)
    # removing unranked players and rankings that have been loaded already
    df_load_rank = df_load_rank.loc[pd.notnull(df_load_rank['rank'])]
    df_load_rank = df_load_rank.loc[df_load_rank['date'] >= pd.to_datetime(today)]



except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)

In [ ]:
if df_missing_players_rank.shape[0] > 0:
    df_missing_players_rank.to_csv('../data/missingPlayersRank.csv')
df_missing_players_rank.shape

In [ ]:
try:
    engine = create_engine(dbConnectionString)
    with engine.connect() as conn:
        
        # insert using SQLalchemy
        data = df_load_rank.replace(np.nan, None).values.tolist()
        rankMeta = sal.MetaData(engine)
        rankTable = sal.Table('ranking', rankMeta, autoload=True)
        conn.execute(sal.insert(rankTable).values(data))
    
    ''' old mysql.connector connection
        query_insert = """
            INSERT INTO ranking 
                (outletId, date, season, week, rankGroup, analystId, ranking, high, low, playerId)
            VALUES 
                (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """

        cursor.executemany(query_insert, df_load_rank.values.tolist())
        conn.commit()
        conn.close()
        '''
    print("success")
except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)

#### INPUT TO CONTROL DB ACTIONS - USED TO MANUALLY ADD PLAYERS MISSING FROM DB

In [ ]:
action = 'update'
source = 'cbs'

db_action = {
            'update':{
                        "espn": """UPDATE player SET espnId = %s,espnName = %s WHERE playerId = %s""",
                         "cbs": """UPDATE player SET cbsId = %s,cbsName = %s WHERE playerId = %s""",
                         "fp": """UPDATE player SET fpId = %s,fpName = %s WHERE playerId = %s""",
                         "nfl": """UPDATE player SET nflId = %s,nflName = %s WHERE playerId = %s"""
                },
             'add':{
                         "espn": """INSERT INTO player (posId,teamId,espnId,espnName,name) VALUES (%s,%s,%s,%s,%s)""",
                         "cbs": """INSERT INTO player (posId,teamId,cbsId,cbsName,name) VALUES (%s,%s,%s,%s,%s)""",
                         "fp": """INSERT INTO player (posId,teamId,fpId,fpName,name) VALUES (%s,%s,%s,%s,%s)""",
                         "nfl": """INSERT INTO player (posId,teamId,nflId,nflName,name) VALUES (%s,%s,%s,%s,%s)"""
             },
            'delete':{
                
            }
}

db_action[action][source]

In [ ]:
#####
## THIS CAN BE USED TO UPDATE THE DB FOR MISSING PLAYERS
#####
df_missing = pd.read_excel(r'../data/adds.xlsx', sheet_name=action)

try:
    #engine = create_engine(dbConnectionString)
    #conn = engine.connect()
    conn = mysql.connector.connect(user=dbUser, password=dbPw,
                              host=dbHost,database=dbName)
    cursor = conn.cursor()
    
    cursor.executemany(db_action[action][source], df_missing.values.tolist())
        
    conn.commit()
    conn.close()
    print("success")
    
except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)

# LOAD PROJECTIONS

In [ ]:
try:  
    projection_columns_dbLoad = ["playerId", "date", "season", "week", "outlet",'GamesPlayed',
     'PassAttempts','PassCompletions','PassingYards', 'PassingYardsPerGame', 'TouchdownsPasses', 
     'InterceptionsThrown', 'PasserRating','RushingAttempts','RushingYards', 'AverageYardsPerRush', 'RushingTouchdowns',
     'Targets', 'Receptions', 'ReceivingYards', 'YardsPerGame', 'AverageYardsPerReception','ReceivingTouchdowns',
     'FumblesLost','FieldGoalsMade','FieldGoalAttempts','LongestFieldGoal','FieldGoals119Yards','FieldGoals119YardAttempts',
     'FieldGoals2029Yards','FieldGoals2029YardAttempts','FieldGoals3039Yards','FieldGoals3039YardAttempts',
     'FieldGoals4049Yards','FieldGoals4049YardAttempts','FieldGoals50Yards','FieldGoals50YardsAttempts',
     'ExtraPointsMade','ExtraPointsAttempted','Interceptions','Safeties','Sacks','Tackles','DefensiveFumblesRecovered',
     'ForcedFumbles','DefensiveTouchdowns', 'ReturnTouchdowns','PointsAllowed','PointsAllowedPerGame','NetPassingYardsAllowed',
     'RushingYardsAllowed','TotalYardsAllowed', 'YardsAgainstPerGame', 'twoPt', 'FantasyPoints','FantasyPointsPerGame']

    projcols = ['outlet', 'date', 'season', 'week', 'sourceId', 'name', 'shortName', 'pos', 'team', 'GamesPlayed', 
                'PassAttempts', 'PassCompletions', 'PassingYards', 'PassingYardsPerGame', 'TouchdownsPasses', 
                'InterceptionsThrown', 'PasserRating', 'RushingAttempts', 'RushingYards', 'AverageYardsPerRush', 
                'RushingTouchdowns', 'Targets', 'Receptions', 'ReceivingYards', 'YardsPerGame', 'AverageYardsPerReception', 
                'ReceivingTouchdowns', 'FumblesLost', 'FieldGoalsMade', 'FieldGoalAttempts', 'LongestFieldGoal', 
                'FieldGoals119Yards', 'FieldGoals119YardAttempts', 'FieldGoals2029Yards', 'FieldGoals2029YardAttempts', 
                'FieldGoals3039Yards', 'FieldGoals3039YardAttempts', 'FieldGoals4049Yards', 'FieldGoals4049YardAttempts', 
                'FieldGoals50Yards', 'FieldGoals50YardsAttempts', 'ExtraPointsMade', 'ExtraPointsAttempted', 'Interceptions', 
                'Safeties', 'Sacks', 'Tackles', 'DefensiveFumblesRecovered', 'ForcedFumbles', 'DefensiveTouchdowns', 
                'ReturnTouchdowns', 'PointsAllowed', 'PointsAllowedPerGame', 'NetPassingYardsAllowed',  'RushingYardsAllowed',
                'TotalYardsAllowed', 'YardsAgainstPerGame', 'twoPt', 'FantasyPoints', 'FantasyPointsPerGame']

    #main df to hold all data
    df_load_proj = pd.DataFrame(columns = projection_columns_dbLoad)
    
    # df to hold players that are not in the database for a source yet
    df_missing_players_proj = pd.DataFrame(columns=['date', 'outlet', 'playerId', 'sourceId', 'name'])

    engine = create_engine(dbConnectionString)
    with engine.connect() as conn:
    
        directory = r'../data/projection/weekly'
        #looping through every ADP file to aggregate into single df
        for filename in os.listdir(directory):
            f = os.path.join(directory,filename)
            # checking if it is a file
            if os.path.isfile(f):
                temp = pd.read_csv(f, parse_dates=['date'], 
                                   names=projcols,
                          skiprows=1)

                # creating dicts to convert outlet name/id to db id
                if 'cbs' in f:

                    # converting full team name to db table 'TEAM'.name
                    lookup = pd.read_sql("SELECT cbsId, name FROM player WHERE cbsId IS NOT NULL;", conn)
                    lookup = pd.Series(lookup.cbsId.values, index=lookup.name).to_dict()

                    # updating the cbs source data full team name to the abbreviated db name
                    temp['team'] = temp['team'].replace("JAC", "JAX").replace("WAS","WSH")
                    temp.loc[temp['sourceId'].isna(), 'sourceId'] = temp.loc[temp['sourceId'].isna(), 'team'].map(lookup)

                    # cbs source data does not have playerId for the defenses
                    lookup = pd.read_sql("SELECT playerId, cbsId FROM player WHERE cbsId IS NOT NULL;", conn)
                    lookup = pd.Series(lookup['playerId'].values, index=lookup.cbsId).to_dict()

                elif 'espn' in f:
                    # espn source data does not have playerId for the defenses
                     # converting full team name to db table 'TEAM'.name
                    lookup = pd.read_sql("SELECT espnId, name FROM player WHERE espnId IS NOT NULL;", conn)
                    lookup = pd.Series(lookup.espnId.values, index=lookup.name).to_dict()

                    # updating the nfl source data full team name to the abbreviated db name
                    temp.loc[temp['sourceId'].isna(), 'sourceId'] = temp.loc[temp['sourceId'].isna(), 'team'].map(lookup)

                    lookup = pd.read_sql("SELECT espnId, playerId FROM player WHERE espnId IS NOT NULL;", conn)
                    lookup = pd.Series(lookup.playerId.values, index=lookup.espnId).to_dict()


                elif 'nfl' in f:
                    # converting full team name to db table 'TEAM'.name
                    lookup = pd.read_sql("SELECT nflName, name FROM team WHERE nflName IS NOT NULL;", conn)
                    lookup = pd.Series(lookup.name.values, index=lookup.nflName).to_dict()

                    # updating the nfl source data full team name to the abbreviated db name
                    temp.loc[temp['name'].isna(), 'team'] = temp.loc[temp['name'].isna(), 'team'].map(lookup)

                    lookup = pd.read_sql("SELECT playerId, nflId FROM player WHERE nflId IS NOT NULL;", conn)
                    lookup = pd.Series(lookup.playerId.values, index=lookup.nflId).to_dict()


                # using the lookup to make the change from outletId to dbId
                temp['playerId'] = temp['sourceId'].map(lookup)

                # creating a df to hold that date for players who are not in the player table for the source
                if temp[pd.isnull(temp['playerId'])].shape[0] > 0:
                    df_missing_players_proj = pd.concat([df_missing_players_proj, 
                                                    temp.loc[pd.isnull(temp['playerId']), ['date', 'outlet', 'playerId', 'sourceId', 'name']]])

                # updating outlet name to db outlet id 
                temp['outlet'] = temp['outlet'].replace(outletLookup)


                temp = temp[projection_columns_dbLoad].replace("-", None).replace("—", None)

                df_load_proj = pd.concat([df_load_proj, temp])

    df_load_proj = df_load_proj.replace(np.nan, None)

    
except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)

In [ ]:
if df_missing_players_proj.shape[0] > 0:
    df_missing_players_proj.to_csv('../data/missingPlayersProj.csv')
df_missing_players_proj.shape

In [ ]:
try:
    engine = create_engine(dbConnectionString)
    with engine.connect() as conn:
        
        # insert using SQLalchemy
        data = df_load_proj.replace(np.nan, None).values.tolist()
        projMeta = sal.MetaData(engine)
        projTable = sal.Table('projection', projMeta, autoload=True)
        conn.execute(sal.insert(projTable).values(data))

        '''  old connection using mysql.connector
        query_insert = """
            INSERT INTO projection 
            VALUES 
                (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                 %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.executemany(query_insert, df_load_proj.values.tolist())
        conn.commit()
        conn.close()
        '''
    print("success")
except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)

# INSERT BETTING LINES - updated

In [3]:
directory = r'../data/betting'
bettingTableName = 'betting'

bettingCols = ['date', 'season', 'week', 'overUnder', 'overUnderCost', 'awayTeamId',
               'awaySpread', 'awayCost', 'awayMoneyline', 'homeTeamId', 'homeSpread',
               'homeCost', 'homeMoneyLine']

df_lines = pd.DataFrame(columns=bettingCols)

#looping through every betting line file to aggregate into single df
for filename in os.listdir(directory):
    f = os.path.join(directory,filename)
    # checking if it is a file
    if os.path.isfile(f):
        temp = pd.read_csv(f, parse_dates=['date'],  names=bettingCols, skiprows=1)
        df_lines = pd.concat([df_lines, temp], ignore_index=True)


try:
    engine = create_engine(dbConnectionString)
    
    with engine.connect() as conn:
        tran = conn.begin()

        lookup = pd.read_sql("SELECT bpName, teamId FROM team;", conn)
        lookup = pd.Series(lookup.teamId.values, index=lookup.bpName).to_dict()
            
        df_lines['awayTeamId'] = df_lines['awayTeamId'].map(lookup)
        df_lines['homeTeamId'] = df_lines['homeTeamId'].map(lookup)

        df_lines.to_sql(bettingTableName, conn, if_exists='append', index=False)

        tran.commit()
        conn.close()

    print("success")
except Exception as ex:
    tran.rollback()
    conn.close()
    print(ex)

C:\Users\jrbrz\AppData\Local\Temp\ipykernel_7260\3401799653.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_lines = pd.concat([df_lines, temp], ignore_index=True)


NameError: name 'tran' is not defined

# OLDER DATA

## INSERTING PLAYER NAMES/ID

In [ ]:
#players = pd.read_excel("Data/names_backup.csv", sheet_name='names').replace(np.nan, None)#, index_col='playerId').replace(np.nan, None)
#updates, adds, delete, Sheet2
players = pd.read_excel("Data/adds.xlsx", sheet_name='Sheet1')
players = players.replace(np.nan, None).replace(0,None)
players.head(2)

In [ ]:
try:
    conn = mysql.connector.connect(user=dbUser, password=dbPw,
                              host=dbHost,database=dbName)
    cursor = conn.cursor()
    
    query_insert = """
        INSERT INTO player 
            (posId, teamId, name, espnId, espnName, bpId, bpName, nflId, nflName, fpId, fpName, cbsId, cbsName)
        VALUES 
            (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """    
    query_update = """
        REPLACE INTO player 
            (playerId, posId, teamId, name, espnId, espnName, bpId, bpName, nflId, nflName, fpId, fpName, cbsId, cbsName)
        VALUES 
            (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """ 
    query_update1 = """
        UPDATE player 
        SET 
            mflId = %s, sportradarId = %s, gsisId = %s, pffId = %s,  sleeperId = %s, yahooId = %s, 
            fleaflickerId = %s, rotowireId = %s, rotoworldId = %s,  ktcId = %s, pfrId = %s, 
            cfbrefId = %s, statsId = %s, statsglobalId = %s, fantasydataId = %s, swishId = %s
        WHERE playerId = %s
    """
    
    query_insert1 = """
        INSERT INTO player 
            (posId, teamId, cbsId, cbsName, name)
        VALUES 
            (%s, %s, %s, %s, %s)
    """
    
    query_delete = '''
        DELETE FROM player
        WHERE playerId = %s
    '''
    
    query_update = '''
        UPDATE projection SET playerID = %s WHERE playerId = %s
    '''
    
    cursor.executemany(query_update1, players.values.tolist())
    conn.commit()
    conn.close()

except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)

## ADP

In [ ]:
try:
    engine = create_engine(dbConnectionString)
    conn = engine.connect()
    
    # columns for db table adp
    adp_columns = ['outlet', 'date', 'playerId', 'adp','high','low']

    #main df to hold all data
    df_load_adp = pd.DataFrame(columns = adp_columns)

    directory = r"..\data\adp"
    #looping through every ADP file to aggregate into single df
    for filename in os.listdir(directory):
        f = os.path.join(directory,filename)
        # checking if it is a file
        if os.path.isfile(f):
            temp = pd.read_excel(f, parse_dates=['date'])

            # creating dicts to convert outlet name/id to db id
            if 'cbs' in f:

                # creating cbs ID to db ID map
                lookup = pd.read_sql("SELECT playerId, cbsId FROM player WHERE cbsId IS NOT NULL;", conn)
                lookup = pd.Series(lookup['playerId'].values, index=lookup.cbsId).to_dict()

            elif 'espn' in f:
                # espn source data does not have playerId for the defenses
                # converting full team name to db table 'TEAM'.name
                lookup = pd.read_sql("SELECT espnId, name FROM player WHERE espnId IS NOT NULL;", conn)
                lookup = pd.Series(lookup.espnId.values, index=lookup.name).to_dict()

                # updating the player table team name to player table Id
                temp['team'] = temp['team'].str.upper()
                temp.loc[temp['playerId'].isna(), 'playerId'] = temp.loc[temp['playerId'].isna(), 'team'].map(lookup)

                lookup = pd.read_sql("SELECT espnId, playerId FROM player WHERE espnId IS NOT NULL;", conn)
                lookup = pd.Series(lookup.playerId.values, index=lookup.espnId).to_dict()


            elif 'fp' in f:

                # updating the fantasy pro source data id to db id
                temp = temp.loc[pd.notnull(temp['adp'])]

                lookup = pd.read_sql("SELECT playerId, fpId FROM player WHERE fpId IS NOT NULL;", conn)
                lookup = pd.Series(lookup.playerId.values, index=lookup.fpId).to_dict()



            # using the lookup to make the change from outletId to dbId
            temp['playerId'] = temp['playerId'].map(lookup)

            # updating outlet name to db outlet id 
            temp['outlet'] = temp['outlet'].replace(outletLookup)


            temp = temp[adp_columns]
            # adding outlet dataframe to the upload dataframe
            df_load_adp = pd.concat([df_load_adp, temp])

    df_load_adp = df_load_adp.replace(np.nan, None)
    conn.close()
    
except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)

In [ ]:
try:
    #engine = create_engine(dbConnectionString)
    #conn = engine.connect()
    conn = mysql.connector.connect(user=dbUser, password=dbPw,
                              host=dbHost,database=dbName)
    cursor = conn.cursor()
    
    query_insert = """
        INSERT INTO adp 

        VALUES 
            (%s, %s, %s, %s, %s, %s)
    """

    cursor.executemany(query_insert, df_load_adp.values.tolist())
    conn.commit()
    conn.close()
    
except Exception as ex:
    conn.rollback()
    conn.close()
    print(ex)